# Neuronexus Analysis v6

**PLX → LFP Filtering → Spike Sorting → Phase Locking**

このノートブックでは、PLXファイルを読み込んでLFP解析・スパイクソーティング・位相ロック解析を順番に実行します。  
各セクションのON/OFFは下の設定セルで切り替えてください。

## 0. 設定

In [1]:
# ============================================================
# 解析設定 - 必要に応じて変更してください
# ============================================================

# PLXファイルパス（空文字列 → GUIで選択）
PLX_FILE = ""

# 出力ディレクトリ（空文字列 → PLXファイルと同じフォルダ）
OUTPUT_DIR = ""

# 各ステップのON/OFF
RUN_LFP    = True    # LFP処理を実行するか
RUN_SPIKE  = True    # スパイクソーティングを実行するか
RUN_PHASE  = True    # 位相ロック解析を実行するか

# Widebandデータを読み込むか（メモリ節約のためFalseにできる）
LOAD_WIDEBAND = True

## 1. PLXファイル読み込み

In [2]:
import os
import sys
import numpy as np

# NumPy 2.0+ 互換パッチ（neo/quantities より先に実行）
import numpy_compat

# Matplotlib をノートブック用に設定
# GUIウィンドウを使うので %matplotlib tk を使用
%matplotlib tk

print("=" * 60)
print("  Neuronexus Analysis v6 (Notebook)")
print("=" * 60)

  Neuronexus Analysis v6 (Notebook)


In [ ]:
# PLXファイル選択
plx_file = PLX_FILE
if not plx_file:
    from get_path import get_path
    plx_file = get_path(mode='file', file_type='plx', init_dir=r'\\Nagoya_AMCBNAS2\wakelab7\okita\whisker_analysis\20260114-1_Wildtype_S1BF\260114-4_Wildtype_S1BF_920_wisker')

if not plx_file or not os.path.exists(plx_file):
    raise FileNotFoundError(f"PLXファイルが見つかりません: {plx_file}")

print(f"\n[Step 0] PLX読み込み: {os.path.basename(plx_file)}")
from data_loader import load_plx
plx_data = load_plx(plx_file, load_wideband=LOAD_WIDEBAND)

if OUTPUT_DIR:
    plx_data.output_dir = OUTPUT_DIR
output_dir = plx_data.output_dir
basename = plx_data.basename

# 結果を保持
lfp_results = None
spike_results = None
phase_results = None

print(f"\n出力先: {output_dir}")

## 2. LFP Filtering

LFP Filter GUI が起動します。`plx_data` のファイルパス・チャンネル情報等は自動でGUIに反映されます。  
フィルタ設定を行い「実行」ボタンを押してください。GUI を閉じると次のセルに進めます。

In [ ]:
if RUN_LFP:
    print("[Step 1] LFP Filter GUI を起動...")

    from lfp_filter_gui import LfpFilterGUI

    lfp_config_holder = [None]

    def on_lfp_done(config, _plx_data):
        lfp_config_holder[0] = config

    # plx_data を渡すと、ファイルパス・データ情報が自動でGUIに反映される
    gui = LfpFilterGUI(plx_data=plx_data, on_done=on_lfp_done)
    gui.run()

    lfp_config = lfp_config_holder[0]
    if lfp_config is None:
        print("  ⚠ LFP処理がキャンセルされました")
    else:
        print("[Step 1b] LFPパイプライン実行...")
        from lfp_pipeline import run_lfp_pipeline
        lfp_results = run_lfp_pipeline(lfp_config, plx_data)
        print("  ✓ LFP処理完了")
else:
    print("[Step 1] LFP処理: スキップ")

## 3. Spike Sorting

Spike Sorting GUI が起動します。各チャンネルのスパイクソーティングを行ってください。

In [ ]:
if RUN_SPIKE and plx_data.wideband_raw is not None:
    print("[Step 2] Spike Sorting GUI を起動...")

    from spike_sort_gui import SpikeSortGUI

    spike_holder = [None]

    def on_spike_done(results):
        spike_holder[0] = results

    gui = SpikeSortGUI(
        wideband_data=plx_data.wideband_raw,
        fs=plx_data.wideband_fs,
        output_dir=output_dir,
        basename=basename,
        on_done=on_spike_done
    )
    gui.run()
    spike_results = spike_holder[0]

    if spike_results:
        from spike_plotting import plot_spike_grand_summary, plot_quality_table
        plot_spike_grand_summary(spike_results, output_dir, basename,
                                 show=True, save=True)
        plot_quality_table(spike_results, output_dir, basename,
                           show=True, save=True)
        n_units = sum(len(r.units) for r in spike_results.values())
        print(f"  ✓ スパイクソーティング完了: {n_units} units")
    else:
        print("  ⚠ スパイクソーティングがキャンセルされました")

elif not RUN_SPIKE:
    print("[Step 2] スパイクソーティング: スキップ")
else:
    print("[Step 2] Widebandデータなし - スパイクソーティングをスキップ")

## 4. Phase Locking Analysis

位相ロック解析GUI が起動します。解析帯域や条件を設定してください。

In [ ]:
if RUN_PHASE and lfp_results is not None:
    print("[Step 3] Phase Locking GUI を起動...")

    condition_masks = None
    if all(k in lfp_results for k in ['clean_baseline', 'clean_stim', 'clean_post']):
        condition_masks = {
            'baseline': lfp_results['clean_baseline'],
            'stim': lfp_results['clean_stim'],
            'post': lfp_results['clean_post']
        }

    from phase_gui import PhaseGUI

    phase_holder = [None, None]

    def on_phase_done(pr, cr):
        phase_holder[0] = pr
        phase_holder[1] = cr

    gui = PhaseGUI(
        lfp_cleaned=lfp_results.get('lfp_cleaned'),
        lfp_times=lfp_results.get('lfp_times'),
        fs=lfp_results.get('fs'),
        segment=plx_data.segment,
        spike_results=spike_results,
        stim_times=lfp_results.get('stim_times'),
        condition_masks=condition_masks,
        original_ch_numbers=lfp_results.get('original_ch_numbers', []),
        output_dir=output_dir,
        basename=basename,
        on_done=on_phase_done
    )
    gui.run()
    phase_results = phase_holder[0]

    if phase_results:
        print("  ✓ 位相ロック解析完了")
    else:
        print("  ⚠ 位相ロック解析がキャンセルされました")

elif not RUN_PHASE:
    print("[Step 3] 位相ロック解析: スキップ")
else:
    print("[Step 3] LFPデータなし - 位相ロック解析をスキップ")

## 5. 完了

In [ ]:
print("=" * 60)
print("  Neuronexus Analysis v6 完了!")
print(f"  出力先: {output_dir}")
print("=" * 60)

print("\n--- 結果サマリー ---")
if lfp_results is not None:
    n_ch = lfp_results.get('lfp_cleaned', np.empty((0,0))).shape[1] if lfp_results.get('lfp_cleaned') is not None else 0
    print(f"  LFP: {n_ch} チャンネル処理済み")
else:
    print("  LFP: 未処理")

if spike_results is not None:
    n_units = sum(len(r.units) for r in spike_results.values())
    print(f"  スパイク: {n_units} units ({len(spike_results)} チャンネル)")
else:
    print("  スパイク: 未処理")

if phase_results is not None:
    print(f"  位相ロック: 解析完了")
else:
    print("  位相ロック: 未処理")